In [1]:
# Autocompletion
%config Completer.use_jedi = False

# Autoreload  
%load_ext autoreload
%autoreload 2

In [2]:
# Autoreload
import sys
from typing import List

sys.path.append('../')

import numpy as np
from IPython.display import Audio, display


import nltk
import torch
import numpy as np
import datasets
import pandas as pd
import transformers

from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel, T5ForConditionalGeneration, AutoModelForSeq2SeqLM

In [3]:
import llmsearch

Monkey Patching .generate function of `transformers` library


In [4]:
device = "cpu"

if torch.backends.mps.is_built() and torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"

from llmsearch.utils.mem_utils import gc_cuda

print(f"Device - {device}")

def beep(duration = 1, frequency=440, rhythm=1):
    sample_rate = 44100  # Standard audio sample rate
    t = np.linspace(0, duration, int(duration * sample_rate), endpoint=False)
    audio_data = np.sin(2*np.pi*frequency*t)  # Generate a sine wave
    audio_data *= np.where(np.arange(len(audio_data)) % rhythm == 0, 1, 0)  # Apply rhythm
    display(Audio(audio_data, rate=sample_rate, autoplay=True))

Device - mps


In [5]:
dataset = datasets.load_dataset("samsum")

Found cached dataset samsum (/Users/praful932/.cache/huggingface/datasets/samsum/samsum/0.0.0/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e)


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
sample_size = 100
samples_to_tune_on = datasets.Dataset.from_dict(dataset["train"][:sample_size])
samples_to_tune_on = samples_to_tune_on.rename_columns(column_mapping = {'dialogue' : 'X', 'summary' : "y"})

In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast = False)
model =  AutoModelForSeq2SeqLM.from_pretrained(model_id).to(device)



In [8]:
import langchain

X = samples_to_tune_on[0]['X']

pt = langchain.PromptTemplate.from_template("Conversation: {X}\nSummary:")

print(pt.format_prompt(X = X).to_string())

Conversation: Amanda: I baked  cookies. Do you want some?
Jerry: Sure!
Amanda: I'll bring you tomorrow :-)
Summary:


In [9]:
import evaluate

rouge_metric = evaluate.load("rouge")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]
    

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels


def get_rouge_score(y_true: List, y_pred: List):
    preds, gts = postprocess_text(preds=y_pred, labels=y_true)

    result = rouge_metric.compute(predictions=preds, references=gts, use_stemmer=True, use_aggregator=False)
    return np.mean(result['rouge2'])

In [10]:
from llmsearch.tuner import Tuner
from llmsearch.utils.mem_utils import get_total_available_ram, get_gpu_information
from llmsearch.utils.logging_utils import set_verbosity_info, set_verbosity_debug
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

seed = 42

set_verbosity_info()


In [11]:
tuner_ob = Tuner(model = model,tokenizer = tokenizer,dataset = samples_to_tune_on,
                 device = device, batch_size = 512,
                 tokenizer_encoding_kwargs={'padding': True, 'truncation': True, 'max_length': 512},
                 tokenizer_decoding_kwargs = {'skip_special_tokens' : True,  'spaces_between_special_tokens' : False}, 
                 scorer = get_rouge_score, prompt_template = pt, is_encoder_decoder = True, seed = seed, column_mapping = {"text_column_name": "X", "label_column_name": "y"})


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [12]:
# Earlier
from llmsearch.utils.model_utils import seed_everything
"""
parameters and how they affect do_sample == False
1. temperature - output does not change - greedy decoding
2. top_k - output does not change - greedy decoding
3. repetition_penalty - output changes
4. no_repeat_ngram_size - output changes
"""

# seed_everything(seed)

initial_generation_params1 = {
    'max_new_tokens' : 120,
    'num_beams': 3,
#     'temperature' : 0.7,
#     'do_sample' : True,
#     'generation_seed' : 42,
#     'tfs' : 0.95,
#     'top_a' : 0.3,
    
#     "epsilon_cutoff": 1.49,
#     "eta_cutoff": 10.42,
#     "repetition_penalty": 1.17,
#     "temperature": 1.31,
#     "top_a": 0.52,
#     "top_k": 49,
#     "top_p": 0.14,
#     "do_sample": True,
#     "generation_seed": 42,
}
score, outputs1 = tuner_ob.get_score(initial_generation_params1)


2023-09-23 15:41:55.558 - llmsearch.utils.mem_utils:145 - INFO - Starting inference with generation parameters - {'max_new_tokens': 120}
2023-09-23 15:41:55.559 - llmsearch.utils.mem_utils:149 - INFO - Performing inference with batch_size - 512
2023-09-23 15:41:55.559 - llmsearch.utils.model_utils:102 - INFO - Detected generation type - Greedy Decoding


  0%|          | 0/1 [00:00<?, ?it/s]

/Users/praful932/Library/Caches/pypoetry/virtualenvs/bhaasha-VBcDHvMD-py3.8/lib/python3.8/site-packages/transformers/generation/utils.py:2389: UserWarning: MPS: no support for int64 min/max ops, casting it to int32 (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/ReduceOps.mm:1271.)
  if unfinished_sequences.max() == 0:
2023-09-23 15:42:23.838 - llmsearch.utils.mem_utils:179 - INFO - Finished running inference, took 28.278377 secs


In [13]:
print(score)

0.18734483930420814


In [ ]:
# Earlier
from llmsearch.utils.model_utils import seed_everything
"""
parameters and how they affect do_sample == False
1. temperature - output does not change - greedy decoding
2. top_k - output does not change - greedy decoding
3. repetition_penalty - output changes
4. no_repeat_ngram_size - output changes
"""

# seed_everything(seed)

gen_params = {
    'do_sample': False, 
    'generation_seed': 42, 
    'max_new_tokens': 120, 
    'no_repeat_ngram_size': 5, 
    'num_beam_groups': 1, 
    'num_beams': 3,
}
score, outputs1 = tuner_ob.get_score(gen_params)


2023-09-23 16:04:56.710 - llmsearch.utils.mem_utils:145 - INFO - Starting inference with generation parameters - {'do_sample': False, 'generation_seed': 42, 'max_new_tokens': 120, 'no_repeat_ngram_size': 5, 'num_beam_groups': 1, 'num_beams': 3}
2023-09-23 16:04:56.711 - llmsearch.utils.mem_utils:149 - INFO - Performing inference with batch_size - 512
2023-09-23 16:04:56.711 - llmsearch.utils.model_utils:102 - INFO - Detected generation type - Beam Search


  0%|          | 0/1 [00:00<?, ?it/s]

/Users/praful932/Library/Caches/pypoetry/virtualenvs/bhaasha-VBcDHvMD-py3.8/lib/python3.8/site-packages/transformers/generation/utils.py:719: UserWarning: MPS: no support for int64 repeats mask, casting it to int32 (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/Repeat.mm:236.)
  input_ids = input_ids.repeat_interleave(expand_size, dim=0)


In [ ]:
print(score)

In [14]:
from llmsearch.utils.gen_utils import get_sample_hyp_space

sample_hyp_spaces = get_sample_hyp_space(seed = 42, max_new_tokens = 120)
sample_hyp_spaces[1]

[{'epsilon_cutoff': [1.49],
  'eta_cutoff': [10.42],
  'repetition_penalty': [1.17],
  'temperature': [1.31],
  'top_a': [0.52],
  'top_k': [49],
  'top_p': [0.14],
  'do_sample': [True],
  'generation_seed': [42],
  'max_new_tokens': [120]},
 {'repetition_penalty': [1.01],
  'temperature': [0.87],
  'tfs': [0.68],
  'top_k': [85],
  'top_p': [0.99],
  'typical_p': [0.68],
  'do_sample': [True],
  'generation_seed': [42],
  'max_new_tokens': [120]},
 {'repetition_penalty': [1.15],
  'temperature': [0.7],
  'top_k': [20],
  'top_p': [0.9],
  'do_sample': [True],
  'generation_seed': [42],
  'max_new_tokens': [120]},
 {'repetition_penalty': [1.09],
  'temperature': [1.31],
  'top_k': [72],
  'top_p': [0.29],
  'do_sample': [True],
  'generation_seed': [42],
  'max_new_tokens': [120]},
 {'temperature': [0.2],
  'top_k': [50],
  'top_p': [0.95],
  'do_sample': [True],
  'generation_seed': [42],
  'max_new_tokens': [120]},
 {'encoder_repetition_penalty': [1.07],
  'eta_cutoff': [10.78],
  '

In [15]:
hyp_param_grid = sample_hyp_spaces[0]

scorer = make_scorer(score_func=get_rouge_score, greater_is_better=True)

clf = GridSearchCV(
    estimator = tuner_ob.estimator,
    param_grid=hyp_param_grid,
    scoring = scorer,
    cv = 5,
    n_jobs = None,
)

# clf = RandomizedSearchCV(
#     estimator=tuner_ob.estimator,
#     param_distributions=hyp_param_grid,
#     n_iter = 2,
#     scoring=scorer,
#     cv=5,
#     random_state = 42,
#     n_jobs=None,
# )

# """
# 5 fold means, whole sample set of 100 examples will be split into 80:20 ratio
# for each hyper_parameter set we have a model f(hyper_params)
#     - we will evaluate this model and get the cross val score (test on each 20 samples 5 times, while training on the rest 80 each time)
#     - we get the score on the quality of hyperparams by evaluating the model with the hyperparams on the unseen 1 fold
# """


In [16]:
# tuner_ob.get_score({'do_sample': True, 'generation_seed': 42, 'max_new_tokens': 120, 'repetition_penalty': 1.01, 'temperature': 0.87, 'tfs': 0.68, 'top_k': 85, 'top_p': 0.99, 'typical_p': 0.68})


In [ ]:
clf.fit(X=tuner_ob.dataset["X"], y=tuner_ob.dataset["y"])

2023-09-19 19:25:20.918 - llmsearch.utils.mem_utils:145 - INFO - Starting inference with generation parameters - {'do_sample': True, 'generation_seed': 42, 'max_new_tokens': 120, 'num_beams': 1, 'penalty_alpha': 0.3, 'top_k': 4}
2023-09-19 19:25:20.918 - llmsearch.utils.mem_utils:149 - INFO - Performing inference with batch_size - 512
2023-09-19 19:25:20.919 - llmsearch.utils.model_utils:102 - INFO - Detected generation type - Sampling


  0%|          | 0/1 [00:00<?, ?it/s]

/Users/praful932/Library/Caches/pypoetry/virtualenvs/bhaasha-VBcDHvMD-py3.8/lib/python3.8/site-packages/transformers/generation/utils.py:719: UserWarning: MPS: no support for int64 repeats mask, casting it to int32 (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/Repeat.mm:236.)
  input_ids = input_ids.repeat_interleave(expand_size, dim=0)
2023-09-19 19:25:31.350 - llmsearch.utils.mem_utils:179 - INFO - Finished running inference, took 10.430458 secs
2023-09-19 19:25:31.382 - llmsearch.utils.mem_utils:145 - INFO - Starting inference with generation parameters - {'do_sample': True, 'generation_seed': 42, 'max_new_tokens': 120, 'num_beams': 1, 'penalty_alpha': 0.3, 'top_k': 4}
2023-09-19 19:25:31.383 - llmsearch.utils.mem_utils:149 - INFO - Performing inference with batch_size - 512
2023-09-19 19:25:31.383 - llmsearch.utils.model_utils:102 - INFO - Detected generation type - Sampling


  0%|          | 0/1 [00:00<?, ?it/s]

2023-09-19 19:25:35.401 - llmsearch.utils.mem_utils:179 - INFO - Finished running inference, took 4.018452 secs
2023-09-19 19:25:35.438 - llmsearch.utils.mem_utils:145 - INFO - Starting inference with generation parameters - {'do_sample': True, 'generation_seed': 42, 'max_new_tokens': 120, 'num_beams': 1, 'penalty_alpha': 0.3, 'top_k': 4}
2023-09-19 19:25:35.438 - llmsearch.utils.mem_utils:149 - INFO - Performing inference with batch_size - 512
2023-09-19 19:25:35.439 - llmsearch.utils.model_utils:102 - INFO - Detected generation type - Sampling


...

  0%|          | 0/1 [00:00<?, ?it/s]

here - 2


2023-09-19 21:03:11.515 - llmsearch.utils.mem_utils:179 - INFO - Finished running inference, took 430.935405 secs
2023-09-19 21:03:11.578 - llmsearch.utils.mem_utils:145 - INFO - Starting inference with generation parameters - {'do_sample': True, 'generation_seed': 42, 'max_new_tokens': 120, 'mirostat_eta': 0.1, 'mirostat_mode': 2, 'mirostat_tau': 6}
2023-09-19 21:03:11.579 - llmsearch.utils.mem_utils:149 - INFO - Performing inference with batch_size - 512
2023-09-19 21:03:11.580 - llmsearch.utils.model_utils:102 - INFO - Detected generation type - Sampling


  0%|          | 0/1 [00:00<?, ?it/s]

here - 2


In [ ]:
1

In [ ]:
clf.best_params_

In [ ]:
score2, outputs2 = tuner_ob.get_score(clf.best_params_)

In [ ]:
score2

In [ ]:
print(score2)

In [ ]:
for gt, out1, out2 in zip(samples_to_tune_on['y'], outputs1, outputs2):
    if out1 != out2:
        print(f"Ground Truth - {gt}")
        print(f"out1 - {out1}")
        print(f"out2 - {out2}\n\n")

In [ ]:
beep(duration = 3)

In [ ]:
clf.best_estimator_.set_params(**clf.best_params_).get_params()


In [ ]:
clf.fi

In [ ]:
from sklearn.base import clone

clone(1)

In [ ]:
dir(clf)

In [ ]:
clf.best_estimator_.get_params()

In [ ]:
clf.best_estimator_.get_params()

In [ ]:
clf.cv_results_

In [ ]:
clf.best_estimator_.get_params()